<p style="color:#153462; 
          font-weight: bold; 
          font-size: 30px; 
          font-family: Gill Sans, sans-serif; 
          text-align: center;">
          Function Calling</p>

<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    OpenAI has fine-tuned the "gpt-3.5-turbo-0613" and "gpt-4-0613" models to accept additional
    arguments through which user can pass in description of functions. If it is relevant, returns
    the name of the function to use along with a Json object with he appropriate input parameters.
    <br><br>
    The latest models (gpt-4o, gpt-4-turbo, and gpt-4o-mini) have been trained to both detect when
    a function should be called (depending on the input) and to respond with JSON that adheres to the
    function signature more closely than previous models.
    <br><br>
    Official Function Calling Doc: <a href="https://platform.openai.com/docs/guides/function-calling">
    Function Calling</a>
</p>

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Common Use Cases</span>

<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    Function calling allows you to more reliably get structured data back from the model.<br>
    For example, you can:
    <ol style="text-align: justify; text-justify: inter-word; font-family: cursive;">
        <li>
            Create assistants that answer questions by calling external APIs <br>
            E.g. Define functions like <code>send_email(to: string, body: string), or get_current_weather(location: string, unit: 'celsius' | 'fahrenheit')</code>
        </li>
        <li>
            Convert natural language into API calls <br>
            E.g. convert "Who are my top customers?" to <br>
            <code> get_customers(min_revenue: int, created_before: string, limit: int) </code>
            and call your internal API
        </li>
        <li>
            Extract structured data from text <br>
            e.g. define a function called <code>extract_data(name: string, birthday: string), or sql_query(query: string)</code>
        </li>
    </ol>
</p>

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Basic Steps for Function Calling</span>

<ol style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    <li>Call the model with the user query and a set of functions defined in the functions parameter.</li>
    <li>The model can choose to call one or more functions; if so, the content will be a stringified
        JSON object adhering to your custom schema (note: the model may hallucinate parameters).</li>
    <li>Parse the string into JSON in your code, and call your function with the provided arguments
        if they exist.</li>
    <li>Call the model again by appending the function response as a new message, and let the model
        summarize the results back to the user.</li>
</ol>

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Function calling behavior</span>

<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    The default behavior for tool_choice is tool_choice: "auto". This lets the model decide
    whether to call functions and, if so, which functions to call.
    <br>
    OpenAI offer three ways to customize the default behavior depending on your use case:<br>
    <ol style="text-align: justify; text-justify: inter-word; font-family: cursive;">
        <li>
            To force the model to always call one or more functions, 
            you can set <code>tool_choice: "required"</code>. The model will
            then select which function(s) to call.
        </li>
        <li>
            To force the model to call only one specific function, you can set
            <code>tool_choice: {"type": "function", "function": {"name": "my_function"}}</code>.
        </li>
        <li>
            To disable function calling and force the model to only generate a user-facing
            message, you can set <code>tool_choice: "none"</code>.
        </li>
    </ol>
</p>

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Setting-Up Environment</span>

In [22]:
from openai import OpenAI
import os
import pprint
import json

In [2]:
client = OpenAI(
  api_key = os.getenv("OPENAI")
)

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Helper Functions</span>

In [3]:
# A dummy function, which returns same weather each time. In real world
# this function might be an API etc..,
def get_current_weather(location: str, unit: str="Fahreheit") -> str:
    """
    Get the current weather in a given location
    """
    weather_info: dict = {
        "location": location,
        "temperature": 72,
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Defining Function</span>

<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    Below is the function(a list) which will pass to an LLM. In the below code, the description
    parameter is important because it tells the model whether to call the function or not.
</p>

In [4]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g Andhra Pradesh, India",
                },
                "unit":{"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required":["location"]
        },
    },
]

### <span style="color:#C738BD; font-weight: bold; font-family: cursive">Calling OpenAI API with Functions</span>

In [14]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Kurnool?"
    }
]

In [15]:
response = client.chat.completions.create(
    model = "gpt-4o",  # You can pick latest model from OpenAI function calling document
    messages = messages,
    functions = functions
)

In [16]:
# Using .__dict__ to print data in a formatted manner
pprint.pp(response.__dict__)

{'id': 'chatcmpl-9wBEv6TMxpyf0F0wmbYWPpbMCd9qf',
 'choices': [Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Kurnool, Andhra Pradesh"}', name='get_current_weather'), tool_calls=None, refusal=None))],
 'created': 1723653257,
 'model': 'gpt-4o-2024-05-13',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_3aa7262c27',
 'usage': CompletionUsage(completion_tokens=20, prompt_tokens=80, total_tokens=100)}


In [17]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Kurnool, Andhra Pradesh"}', name='get_current_weather'), tool_calls=None, refusal=None)

In [18]:
response.choices[0].message.function_call

FunctionCall(arguments='{"location":"Kurnool, Andhra Pradesh"}', name='get_current_weather')

In [27]:
args = eval(response.choices[0].message.function_call.arguments)
args

{'location': 'Kurnool, Andhra Pradesh'}

In [29]:
get_current_weather(args["location"])

'{"location": "Kurnool, Andhra Pradesh", "temperature": 72, "unit": "Fahreheit", "forecast": ["sunny", "windy"]}'

In [30]:
# If you any ask irrelevant question then it wouldn't return function
messages = [
    {
        "role": "user",
        "content": "When Harappan Civilization started?"
    }
]

response = client.chat.completions.create(
    model = "gpt-4o",  # You can pick latest model from OpenAI function calling document
    messages = messages,
    functions = functions
)
pprint.pp(response.__dict__)

{'id': 'chatcmpl-9wBMQBELUQqwQJAoqHTJkrRCSMEv7',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The Harappan Civilization, also known as the Indus Valley Civilization, is one of the world's earliest urban civilizations. It started around 2600 BCE and thrived until about 1900 BCE. The civilization was located in the northwestern regions of South Asia, primarily in present-day Pakistan and northwest India. It is renowned for its advanced urban planning, architecture, and social organization.", role='assistant', function_call=None, tool_calls=None, refusal=None))],
 'created': 1723653722,
 'model': 'gpt-4o-2024-05-13',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_3aa7262c27',
 'usage': CompletionUsage(completion_tokens=79, prompt_tokens=78, total_tokens=157)}


<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    If you notice carefully, in the above response <b>content</b> is not None and it return actual response for your question and <b>function_call</b> is None.
</p>

#### <span style="color:#40A578; font-weight: bold; font-family: cursive">Forcing Function call</span>

<p style="text-align: justify; text-justify: inter-word; font-family: cursive;">
    No matter wheather you will ask question related to function are not, but it always calls the function.
</p>

In [32]:
messages = [
    {
        "role": "user",
        "content": "When Harappan Civilization started?"
    }
]

response = client.chat.completions.create(
    model = "gpt-4o",  # You can pick latest model from OpenAI function calling document
    messages = messages,
    functions = functions,
    tool_choice = "required"
)
pprint.pp(response.__dict__)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'tool_choice': 'tool_choice' is only allowed when 'tools' are specified.", 'type': 'invalid_request_error', 'param': 'tool_choice', 'code': None}}